In [60]:
import pandas as pd
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [61]:
with open('../data/random_forest_hyper_params.json','r') as file:
    rf_hyper_params = json.load(file)

In [62]:
X_train = pd.read_csv('../data/x_train_clean.csv', sep='$', decimal=".", engine='python') 
y_train = pd.read_csv('../data/y_train_clean.csv', sep='$', decimal=".", engine='python')   

del X_train['Unnamed: 0']
del y_train['Unnamed: 0']

In [63]:
df = pd.read_csv('../data/test.csv', sep='[,$]' , decimal=".", engine='python')

df.columns = df.columns.str.lower()

df.rename(columns={
    'id': 'id',
    'gender': 'gender',
    'age': 'age',
    'driving_license': 'driving_license',
    'region_code': 'region_code',
    'previously_insured': 'previously_insured',
    'vehicle_age': 'vehicle_age',
    'vehicle__damage': 'vehicle_damage',
    'annual__premium': 'annual_premium',
    'policy_sales_channel': 'policy_sales_channel',
    'vintage': 'vintage'
},
    inplace=True)


# def map_categorials(df):

#     LE = LabelEncoder()
#     df['region_code'] = LE.fit_transform(df.loc[:,'region_code'])
#     df['gender'] = LE.fit_transform(df.loc[:,'gender'])
#     df['vehicle_age'] = LE.fit_transform(df.loc[:,'vehicle_age'])
#     df['vehicle_damage'] = LE.fit_transform(df.loc[:,'vehicle_damage'])
#     df['previously_insured'] = LE.fit_transform(df.loc[:,'previously_insured'])

#     return df

# df_label_encoded = map_categorials(df)

df_one_hot_encoded_data = pd.get_dummies(df, columns = ['driving_license', 'gender', 'vehicle_age', 'vehicle_damage','previously_insured'])

df_one_hot_encoded_data.rename(columns={
    'id': 'id',
    'gender': 'gender',
    'age': 'age',
    'driving_license_0': 'driving_license_No',
    'driving_license_1': 'driving_license_Yes',
    'region_code': 'region_code',
    'previously_insured_0': 'previously_insured_No',
    'previously_insured_1': 'previously_insured_Yes',
    'vehicle_age': 'vehicle_age',
    'vehicle_damage': 'vehicle_damage',
    'annual_premium': 'annual_premium',
    'policy_sales_channel': 'policy_sales_channel',
    'vintage': 'vintage'
},
    inplace=True)

df_one_hot_encoded_data.head()

,id,age,region_code,annual_premium,policy_sales_channel,vintage,driving_license_No,driving_license_Yes,gender_Female,gender_Male,vehicle_age_1-2 Year,vehicle_age_< 1 Year,vehicle_age_> 2 Years,vehicle_damage_No,vehicle_damage_Yes,previously_insured_No,previously_insured_Yes
0,381000,68,28.0,53066.0,12.0,195,0,1,0,1,1,0,0,1,0,0,1
1,381001,78,8.0,28301.0,124.0,195,0,1,0,1,1,0,0,0,1,1,0
2,381002,22,28.0,2630.0,153.0,170,0,1,0,1,0,1,0,1,0,0,1
3,381003,20,28.0,38627.0,124.0,62,0,1,1,0,1,0,0,0,1,1,0
4,381004,44,28.0,24984.0,152.0,255,0,1,1,0,1,0,0,1,0,0,1


In [64]:
rfc = RandomForestClassifier(**rf_hyper_params, n_jobs=-1)

X = df_one_hot_encoded_data.drop('id', axis=1)

random_forest_classifier = rfc.fit(X_train, y_train['response'])

y_pred = rfc.predict(X)

In [65]:
df_y_pred = pd.DataFrame(y_pred, columns=['y_pred'])

res = df_y_pred.set_index(df['id']).reset_index()
res['y_pred'].value_counts(normalize=True)

0    0.995415
1    0.004585
Name: y_pred, dtype: float64